In [69]:
import pandas as pd
import re

ct14 = r"C:\Users\mkreidler\Desktop\35w\14-0436.xlsx"
ct19 = r"C:\Users\mkreidler\Desktop\35w\19-9999.xlsx"


ct14 = pd.read_excel(ct14)
ct19 = pd.read_excel(ct19)

ct19.head()


,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta
0,6S_D5_2,3B4X,HP,NaN,35WS-3B4X,1,0,NaN,NaN
1,6S_D5_1,4A2,HP,35BSIM,35E-23,1,0,PRESS,NaN
2,3E_1,3C10Z,EXT,NaN,35WS-3C10Z,1,0,NaN,NaN
3,3E_1,3B11A,HP,NaN,35WS-3B11A,1,0,NaN,NaN
4,9W_2,6A16A,HP,NaN,35WS-6A16A,1,0,FINIS,NaN


In [70]:
ct14.head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta
0,ZEXTRA,8C7,RAM,NaN,35-68-1,1,0,FINIS,NaN
1,ZEXTRA,8C7,RAM,NaN,35-68-1,1,0,FINIS,NaN
2,ZEXTRA,8C7,RAM,NaN,35-68-1,1,0,FINIS,NaN
3,ZEXTRA,8C7,RAM,NaN,35-68-1,1,0,FINIS,NaN
4,ZEXTRA,8C7,RAM,NaN,35-68-1,1,0,FINIS,NaN


In [71]:
ct14["shipped_14"] = None
ct14["crated_14"] = None


def find_shipped(x):
    if x in ["SHIPPED", "shipped"]:
        return 1
    else:
        return 0
    
def find_crated(x):
    if x in ["CRATED", "crated"]:
        return 1
    else:
        return 0

ct14.shipped_14 = ct14["Current Shipping Sta"].apply(find_shipped)
ct14.crated_14 = ct14["Current Shipping Sta"].apply(find_crated)

ct14_counts = ct14.groupby("Block ID").sum().reset_index()
ct14_counts = ct14_counts[["Block ID", "UnitQuantity", "shipped_14", "crated_14"]]
ct14_counts.rename(columns={"Block ID": "name", "UnitQuantity":"qty_14"}, inplace=True)

ct14_counts.head()


,name,qty_14,shipped_14,crated_14
0,2B1,3,1,1
1,2B10,1,1,0
2,2B11,2,1,0
3,2B12,1,1,0
4,2B13,1,1,0


In [72]:
ct19["shipped_19"] = None
ct19["crated_19"] = None


def find_shipped(x):
    if x in ["SHIPPED", "shipped"]:
        return 1
    else:
        return 0
    
def find_crated(x):
    if x in ["CRATED", "crated"]:
        return 1
    else:
        return 0

ct19.shipped_19 = ct19["Current Shipping Sta"].apply(find_shipped)
ct19.crated_19 = ct19["Current Shipping Sta"].apply(find_crated)

ct19_counts = ct19.groupby("Block ID").sum().reset_index()
ct19_counts = ct19_counts[["Block ID", "UnitQuantity", "shipped_19", "crated_19"]]
ct19_counts.rename(columns={"Block ID": "name", "UnitQuantity":"qty_19"}, inplace=True)

ct19_counts.head()

,name,qty_19,shipped_19,crated_19
0,2B1,4,0,1
1,2B10,2,0,0
2,2B11,1,0,0
3,2B12,1,0,0
4,2B13,1,0,0


In [81]:
merge = ct19_counts.merge(ct14_counts, on="name", how="outer").fillna(0)
merge["ctqty_delta19-14"] = merge.qty_19 - merge.qty_14
merge["ctship_delta19-14"] = merge.shipped_19 - merge.shipped_14
merge["ctcrated_delta19-14"] = merge.crated_19 - merge.crated_14
merge.head()

,name,qty_19,shipped_19,crated_19,qty_14,shipped_14,crated_14,ctqty_delta19-14,ctship_delta19-14,ctcrated_delta19-14
0,2B1,4.0,0.0,1.0,3.0,1.0,1.0,1.0,-1.0,0.0
1,2B10,2.0,0.0,0.0,1.0,1.0,0.0,1.0,-1.0,0.0
2,2B11,1.0,0.0,0.0,2.0,1.0,0.0,-1.0,-1.0,0.0
3,2B12,1.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,0.0
4,2B13,1.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,0.0


In [74]:
ct19_pshipped = ct19[["Block ID", "Priority"]][ct19.Priority == "SHIPPED"].groupby("Block ID").count().reset_index()

ct19_pshipped = ct19_pshipped.rename(columns = {"Block ID":"name", "Priority":"priority_shipped_19"})
ct19_pshipped.head()



,name,priority_shipped_19
0,4B1B,8
1,4B1C,3
2,4B1E,1
3,4B2A,1
4,4B2B,1


In [90]:
merge = merge.merge(ct19_pshipped, on="name", how="outer").fillna(0)
merge[["name","priority_shipped_19", "shipped_19", "shipped_14"]][merge.priority_shipped_19 != 0]

merge["delta"] = merge.priority_shipped_19 - merge.shipped_14

merge[["name","priority_shipped_19", "shipped_19", "shipped_14", "delta"]][merge.delta != 0]

merge.to_excel(r"C:\Users\mkreidler\Desktop\35w\ctrac merges.xlsx")